In [0]:
!pip install keras-tuner
import sys
import pickle
import argparse
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner import HyperModel
import keras.metrics

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=3f3ee781736c30965534469711640ccc221e3838eb23d88caa89f8dcfecb50e6
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=8ced607869bf38cec3b92b0d25ac315f14752aab2170e2dad007ce3fd025f0af
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


Using TensorFlow backend.


In [0]:
df = pickle.load(open('reviews_10000_processed.p', 'rb'))
vocab = pickle.load(open('reviews_10000_vocab.p', 'rb'))

In [0]:
model_fp = 'reviews_10000_tuned_cnn.p'
NAME = 'tune_10000'

CAT_COL = 'stars'

num_reviews = len(df.index)

train_num = int(num_reviews * .8)

train = df.iloc[:train_num]
validate = df.iloc[train_num:]

X_train = train['text'].values
y_train = to_categorical(train[CAT_COL].values)
print(y_train.shape)

X_validate = validate['text'].values
y_validate = to_categorical(validate[CAT_COL].values)
print(y_validate.shape)

max_length = max(max([len(review) for review in X_train]),  
              max([len(review) for review in X_validate]))

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

num_words = len(tokenizer.word_index) + 1

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_validate_tokens = tokenizer.texts_to_sequences(X_validate)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_validate_pad = pad_sequences(X_validate_tokens, maxlen=max_length, padding='post')

# practitioners guide + that one article
#https://keras-team.github.io/keras-tuner/
def build_model(hp):
  global num_words
  model = Sequential()
  model.add(Embedding(num_words, 
                      output_dim=hp.Choice('embedding_output_dimension', values=[50, 100, 150], default=100),
                      input_length=max_length)) # experiment with size of vector space
  model.add(Conv1D(filters=hp.Choice('num_filters', values=[50, 100, 200, 400, 600],default=50),
                    kernel_size=hp.Int('conv_kernel', min_value=1, max_value=10, step=1), 
                    activation=hp.Choice('conv_activ', values=['relu', 'tanh'], default='relu'))) # experiement with filter / kernel size / activation fxn
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dropout(rate=hp.Float('drop_rate', min_value=0, max_value=.5, step=.1)))
  model.add(Dense(6, activation='softmax')) # This is a necessity
  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=[keras.metrics.Precision(name='precision'),
                  keras.metrics.Recall(name='recall')])
  return model

tuner = RandomSearch(
          build_model,
           metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')],
          objective=kerastuner.Objective('val_precision', direction='max'),
          max_trials=5,
          executions_per_trial=1,
          seed=10,
          directory='tuning_out',
          project_name=NAME)

tuner.search_space_summary()

tuner.search(X_train_pad, y_train, epochs=10, validation_data=(X_validate_pad, y_validate))

tuner.results_summary()

best_model = tuner.get_best_models(num_models=1)[0]
model_json = best_model.to_json()
open('reviews_10000_trained_cnn.json', 'w').write(model_json)
best_model.save_weights('reviews_10000_trained_cnn_weights.h5')

(8000, 6)
(2000, 6)


Epoch 1/10
250/250 [==============================] - 19s 76ms/step - loss: 1.4579 - precision: 0.4556 - recall: 0.2329 - val_loss: 0.9811 - val_precision: 0.6274 - val_recall: 0.3506
Epoch 2/10
250/250 [==============================] - 19s 76ms/step - loss: 0.7780 - precision: 0.6811 - recall: 0.4255 - val_loss: 0.9246 - val_precision: 0.7083 - val_recall: 0.4705
Epoch 3/10
250/250 [==============================] - 19s 75ms/step - loss: 0.5067 - precision: 0.7409 - recall: 0.5181 - val_loss: 1.0445 - val_precision: 0.7602 - val_recall: 0.5523
Epoch 4/10
250/250 [==============================] - 19s 76ms/step - loss: 0.2540 - precision: 0.7849 - recall: 0.5919 - val_loss: 1.2291 - val_precision: 0.8016 - val_recall: 0.6238
Epoch 5/10
250/250 [==============================] - 19s 76ms/step - loss: 0.0982 - precision: 0.8176 - recall: 0.6532 - val_loss: 1.4704 - val_precision: 0.8300 - val_recall: 0.6784
Epoch 6/10
250/250 [==============================] - 19s 76ms/step - loss: 0.03

Epoch 1/10
250/250 [==============================] - 8s 31ms/step - loss: 1.2131 - precision: 0.8741 - recall: 0.7739 - val_loss: 0.9697 - val_precision: 0.8684 - val_recall: 0.7568
Epoch 2/10
250/250 [==============================] - 7s 29ms/step - loss: 0.8310 - precision: 0.8640 - recall: 0.7474 - val_loss: 0.9332 - val_precision: 0.8593 - val_recall: 0.7382
Epoch 3/10
250/250 [==============================] - 7s 29ms/step - loss: 0.6790 - precision: 0.8567 - recall: 0.7328 - val_loss: 1.0107 - val_precision: 0.8534 - val_recall: 0.7283
Epoch 4/10
250/250 [==============================] - 7s 30ms/step - loss: 0.5561 - precision: 0.8518 - recall: 0.7269 - val_loss: 1.0697 - val_precision: 0.8497 - val_recall: 0.7253
Epoch 5/10
250/250 [==============================] - 7s 29ms/step - loss: 0.4239 - precision: 0.8496 - recall: 0.7263 - val_loss: 1.1827 - val_precision: 0.8492 - val_recall: 0.7272
Epoch 6/10
250/250 [==============================] - 7s 29ms/step - loss: 0.2987 - p

Epoch 1/10
250/250 [==============================] - 12s 50ms/step - loss: 1.2998 - precision: 0.8584 - recall: 0.7523 - val_loss: 0.9624 - val_precision: 0.8557 - val_recall: 0.7439
Epoch 2/10
250/250 [==============================] - 12s 48ms/step - loss: 0.8308 - precision: 0.8537 - recall: 0.7390 - val_loss: 0.9532 - val_precision: 0.8514 - val_recall: 0.7348
Epoch 3/10
250/250 [==============================] - 12s 48ms/step - loss: 0.6547 - precision: 0.8503 - recall: 0.7326 - val_loss: 0.9568 - val_precision: 0.8489 - val_recall: 0.7305
Epoch 4/10
250/250 [==============================] - 12s 49ms/step - loss: 0.4589 - precision: 0.8490 - recall: 0.7307 - val_loss: 1.0566 - val_precision: 0.8488 - val_recall: 0.7307
Epoch 5/10
250/250 [==============================] - 12s 49ms/step - loss: 0.2720 - precision: 0.8499 - recall: 0.7325 - val_loss: 1.1926 - val_precision: 0.8506 - val_recall: 0.7343
Epoch 6/10
250/250 [==============================] - 12s 49ms/step - loss: 0.13

Epoch 1/10
250/250 [==============================] - 9s 36ms/step - loss: 1.2075 - precision: 0.8612 - recall: 0.7554 - val_loss: 0.9629 - val_precision: 0.8595 - val_recall: 0.7497
Epoch 2/10
250/250 [==============================] - 9s 35ms/step - loss: 0.8269 - precision: 0.8581 - recall: 0.7462 - val_loss: 0.9312 - val_precision: 0.8566 - val_recall: 0.7430
Epoch 3/10
250/250 [==============================] - 9s 35ms/step - loss: 0.6751 - precision: 0.8556 - recall: 0.7410 - val_loss: 0.9882 - val_precision: 0.8545 - val_recall: 0.7391
Epoch 4/10
250/250 [==============================] - 9s 35ms/step - loss: 0.5419 - precision: 0.8540 - recall: 0.7383 - val_loss: 1.0689 - val_precision: 0.8533 - val_recall: 0.7376
Epoch 5/10
250/250 [==============================] - 9s 35ms/step - loss: 0.4077 - precision: 0.8534 - recall: 0.7378 - val_loss: 1.2232 - val_precision: 0.8532 - val_recall: 0.7381
Epoch 6/10
250/250 [==============================] - 9s 36ms/step - loss: 0.2753 - p

Epoch 1/10
250/250 [==============================] - 10s 39ms/step - loss: 1.2378 - precision: 0.8585 - recall: 0.7494 - val_loss: 0.9754 - val_precision: 0.8574 - val_recall: 0.7449
Epoch 2/10
250/250 [==============================] - 10s 39ms/step - loss: 0.8439 - precision: 0.8564 - recall: 0.7423 - val_loss: 0.9216 - val_precision: 0.8552 - val_recall: 0.7398
Epoch 3/10
250/250 [==============================] - 10s 39ms/step - loss: 0.6956 - precision: 0.8546 - recall: 0.7383 - val_loss: 0.9722 - val_precision: 0.8536 - val_recall: 0.7367
Epoch 4/10
250/250 [==============================] - 10s 38ms/step - loss: 0.5804 - precision: 0.8532 - recall: 0.7359 - val_loss: 1.0247 - val_precision: 0.8526 - val_recall: 0.7351
Epoch 5/10
250/250 [==============================] - 10s 39ms/step - loss: 0.4791 - precision: 0.8525 - recall: 0.7351 - val_loss: 1.1033 - val_precision: 0.8521 - val_recall: 0.7349
Epoch 6/10
250/250 [==============================] - 10s 38ms/step - loss: 0.38

INFO:tensorflow:Oracle triggered exit


In [0]:
max_length

2925

In [0]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2925, 150)         257100    
_________________________________________________________________
conv1d (Conv1D)              (None, 2921, 400)         300400    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1460, 400)         0         
_________________________________________________________________
flatten (Flatten)            (None, 584000)            0         
_________________________________________________________________
dropout (Dropout)            (None, 584000)            0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3504006   
Total params: 4,061,506
Trainable params: 4,061,506
Non-trainable params: 0
______________________________________________

In [0]:
best_model.predict_classes(X_train_pad)

AttributeError: ignored

In [0]:
pickle.dump(X_train_pad, open('reviews_10000_X_train_pad.p', 'wb'))

In [0]:
test = pickle.load(open('reviews_10000_test_processed.p', 'rb'))
X_test_tokens = tokenizer.texts_to_sequences(test['text'].values)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')
pickle.dump(X_test_pad, open('reviews_10000_train_10000_test_pad.p', 'wb'))

In [0]:
from sklearn.metrics import classification_report
test_report = classification_report(test['stars'], best_model.predict_classes(X_test_pad), output_dict=True)
train_report = classification_report(train['stars'], best_model.predict_classes(X_train_pad), output_dict=True)
open('cnn_10000_report.json', 'w').close()
print(test_report, file=open('cnn_10000_report.json', 'a'))
print(train_report, file=open('cnn_10000_report.json', 'a'))